In [1]:
from typing import List
import os
import psycopg
from dotenv import load_dotenv
import ollama


In [2]:
load_dotenv("../src/.env")

db_connection_str = (
    f"dbname={os.getenv('DB_NAME')} "
    f"user={os.getenv('DB_USER')} "
    f"password={os.getenv('DB_PASSWORD')} "
    f"host={os.getenv('DB_HOST')} "
    f"port={os.getenv('DB_PORT')}"
)

print("Configuration PostgreSQL chargée ✔️")


Configuration PostgreSQL chargée ✔️


In [3]:
from typing import List
import os

def load_all_transcripts(folder_path: str) -> List[str]:
    corpus_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)

            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    lines = f.read().split("\n")
            except:
                with open(file_path, "r", encoding="latin-1") as f:
                    lines = f.read().split("\n")

            cleaned = [
                l.strip()
                for l in lines
                if l.strip() != "" and not l.startswith("<")
            ]

            corpus_list.extend(cleaned)

    return corpus_list


conversation_folder = "../data/TRANS_TXT/"
corpus_list = load_all_transcripts(conversation_folder)

print("Nombre total de lignes chargées :", len(corpus_list))
corpus_list[:10]


Nombre total de lignes chargées : 1062


['h: U B S bonjour',
 "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e",
 'h: je vous écoute',
 "c: c'est pour",
 "c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol",
 'h: e ce serait pour vous vous souhaiteriez',
 'h: non',
 "c: non non c'est pas pour moi",
 'c: ce serait pour ma fille',
 'h: oui']

In [4]:
from typing import List

def calculate_embeddings_ollama(text: str, model: str = "mxbai-embed-large") -> List[float]:
    response = ollama.embeddings(
        model=model,
        prompt=text
    )
    return response["embedding"]


In [5]:
# test
calculate_embeddings_ollama("bonjour")[0:10]


[0.3945574462413788,
 -0.1477288007736206,
 -0.7611244916915894,
 0.31378623843193054,
 -0.38164928555488586,
 -0.12771975994110107,
 0.6281507611274719,
 0.27708899974823,
 0.23378583788871765,
 0.36259424686431885]

In [6]:
conn = psycopg.connect(db_connection_str)
cur = conn.cursor()

print("Connecté à PostgreSQL :", conn.info.dbname)


Connecté à PostgreSQL : rag_chatbot


In [7]:
cur.execute("DROP TABLE IF EXISTS embeddings;")
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

cur.execute("""
    CREATE TABLE embeddings (
        id SERIAL PRIMARY KEY,
        corpus TEXT,
        embedding vector(1024)
    );
""")

conn.commit()
print("Table embeddings recréée avec VECTOR(1024) ✔️")


Table embeddings recréée avec VECTOR(1024) ✔️


In [8]:
def save_embedding(cursor, corpus: str, embedding: List[float]):
    cursor.execute(
        "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
        (corpus, embedding)
    )

for i, text in enumerate(corpus_list):
    emb = calculate_embeddings_ollama(text)  # maintenant mxbai-embed-large (1024)
    save_embedding(cur, text, emb)

    if i % 50 == 0:
        print(f"{i} lignes insérées…")

conn.commit()
print("🎉 Tous les embeddings ont été insérés avec succès !")


0 lignes insérées…
50 lignes insérées…
100 lignes insérées…
150 lignes insérées…
200 lignes insérées…
250 lignes insérées…
300 lignes insérées…
350 lignes insérées…
400 lignes insérées…
450 lignes insérées…
500 lignes insérées…
550 lignes insérées…
600 lignes insérées…
650 lignes insérées…
700 lignes insérées…
750 lignes insérées…
800 lignes insérées…
850 lignes insérées…
900 lignes insérées…
950 lignes insérées…
1000 lignes insérées…
1050 lignes insérées…
🎉 Tous les embeddings ont été insérés avec succès !


In [14]:
def to_pgvector(x: List[float]) -> str:
    return "[" + ", ".join(str(v) for v in x) + "]"

def search_similar(text: str, k: int = 15):
    emb = calculate_embeddings_ollama(text)
    emb_vec = to_pgvector(emb)

    cur.execute("""
        SELECT id, corpus, embedding <-> %s AS distance
        FROM embeddings
        ORDER BY distance ASC
        LIMIT %s;
    """, (emb_vec, k))

    return cur.fetchall()


In [15]:
conn.rollback()
print("Transaction PostgreSQL réinitialisée ✔️")


Transaction PostgreSQL réinitialisée ✔️


In [16]:
query = "Est-ce que la fac est ouverte pendant l'été ?"
results = search_similar(query, k=15)

for r in results:
    print("ID:", r[0])
    print("Texte:", r[1])
    print("Distance:", r[2])
    print("---")


ID: 235
Texte: c: e c'est ouvert aujourd'hui par exemple
Distance: 13.069447399269245
---
ID: 237
Texte: c: aujourd'hui c'est ouvert par exemple
Distance: 13.627479257660557
---
ID: 662
Texte: c: e il doit y avoir des supports non
Distance: 14.001935552585222
---
ID: 714
Texte: c: dans un instant peut être
Distance: 14.07545735512205
---
ID: 1007
Texte: c: ah pas cette
Distance: 14.116770279529238
---
ID: 317
Texte: c: et de tout ça
Distance: 14.163723149892364
---
ID: 83
Texte: c: à l'accueil
Distance: 14.1980575737383
---
ID: 608
Texte: c: il répond pas
Distance: 14.258205260001278
---
ID: 17
Texte: h: oui alors e la fac de e de lettre et de langues se trouve à Lorient donc il faudrait plutôt  voir avec Lorient pour e savoir si ils organisent des stages mais en tout cas fac est fermée du 23 juillet au 23 août
Distance: 14.261680768060899
---
ID: 21
Texte: c: un organisme sur Vannes qui qui s'occupe de ce genre de chose
Distance: 14.316190418734614
---
ID: 1041
Texte: c: et je voulais

In [22]:
def generate_answer(question: str, retrieved_texts: List[str]) -> str:
    context = "\n---\n".join(retrieved_texts)

    prompt = f"""
Tu es un assistant STRICTEMENT basé sur les documents.

RÈGLES IMPORTANTES :
- Tu NE DOIS JAMAIS inventer d’information.
- Si la réponse n’est pas explicitement dans le contexte,
  tu DOIS répondre : "Information insuffisante dans le corpus."

DOCUMENTS :
{context}

QUESTION :
{question}

RÉPONSE (sans rien inventer) :
"""

    response = ollama.generate(model="llama3.1:latest", prompt=prompt)
    return response["response"]


In [24]:
question = "Est-ce que l’accueil peut transférer un appel vers le secrétariat ?"


top_docs = [r[1] for r in search_similar(question, 16)]

# générer la réponse
answer = generate_answer(question, top_docs)

print("QUESTION :", question)
print("DOCUMENTS UTILISÉS :", top_docs)
print("\nRÉPONSE DU MODÈLE :\n", answer)


QUESTION : Est-ce que l’accueil peut transférer un appel vers le secrétariat ?
DOCUMENTS UTILISÉS : ["c: à l'accueil", "h: si si mais à l'accueil", "c: d'accord je rappelle à l'accueil ok merci", "h: ouais ouais ouais ben e et a dix si il y a personne à l'accueil elle va directement au secrétariat", "h: ah oui c'est retombé à l'accueil", 'c: e donc e s(oit) soit adjoint administratif ou rédacteur e', 'h: e oui je vais vous passer le secrétariat', "h: voilà c'est le Service Universitaire de l'Information et de l'Orientation parce que ici vous êtes à l'accueil de la fac de droit donc e", 'h: concernée donc secrétariat', 'h: ça va la connexion va ré(ouvir) se réouvrir à la e pour la val(idation) pour le e la confirmation du bac', "c: est-ce qu'il y a une ligne directe pour le son service", 'c: e il travaille e au niveau informatique', "c: e oui bonjour j'aurais souhaité avoir le secrétariat de du D U e carrière juridique s'il vous plait", 'h: e attendez si elle est dans son bureau là elle